In [ ]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
# ---------------------------
# 1. Load the ABF file
# ---------------------------
# Replace "file.abf" with the path to your ABF file
abf = pyabf.ABF("/Path/to/Abf/file/file.abf")

abf.setSweep(1)
signal = abf.sweepY   # Ionic current (e.g., in pA)
time = abf.sweepX   # Time vector (e.g., in seconds)

#signal_data = abf.data[0]


In [ ]:
print("Sampling Rate:", abf.dataRate, "Hz")

print("Number of Sweeps:", abf.sweepCount)

print("Protocol Name:", abf.protocol)

print("Recording Start Time:", abf.abfDateTime)

print("Channel Units:", abf.adcUnits)

print("Channel Scaling Factors:", abf.adcUnits)

In [ ]:
from pyabf.tools.abfHeaderDisplay import abfInfoPage

info_page = abfInfoPage(abf)
header_text = info_page.getText()  
print(header_text)

## Single Sweep

In [ ]:
# ---------------------------
# 2. Plot the ionic current trace for single sweep
# ---------------------------
plt.figure(figsize=(10, 4))
plt.plot(time, signal, label="Ionic current")
plt.xlabel("Time (s)")
plt.ylabel("Current (pA)")
plt.title("Ionic Current Trace")
plt.legend()
plt.show()

# All Sweep Load

In [ ]:
all_sweeps = []
all_times = []
for sweep in range(346):
    abf.setSweep(sweep)
    all_sweeps.append(abf.sweepY)
    all_times.append(abf.sweepX)

data = np.concatenate(all_sweeps)
time = np.concatenate(all_times)

### Clustering Events

In [ ]:
data2 = data.reshape(-1, 1)
print("Length of data", len(data))
n_clusters = 2

# Apply k-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(data2)

# Get cluster labels
labels = kmeans.labels_

# Get the cluster centroids
centroids = kmeans.cluster_centers_
print(centroids)

# Get points in each cluster
cluster_1 = data2[labels == 0]
cluster_2 = data2[labels == 1]
# plot_cluster_histogram(cluster_1, cluster_2)
cluster1_indices = np.where(kmeans.labels_ == 0)[0]
cluster2_indices = np.where(kmeans.labels_ == 1)[0]

In [ ]:
cluster_1_flat = np.array([val[0] for val in cluster_1])
cluster_2_flat = np.array([val[0] for val in cluster_2])
plt.figure()
plt.hist(cluster_1_flat, bins=100, color='blue', alpha=0.7, label='Cluster 1')
plt.hist(cluster_2_flat, bins=100, color='red', alpha=0.7, label='Cluster 2')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Distribution of Cluster 1 and Cluster 2')
plt.legend()
plt.show()

### Analysis based on threshold

In [ ]:
# -------------------------------
# 3. Define threshold and event detection parameters
# -------------------------------
threshold = 180

sample_rate = abf.dataRate

min_event_duration = 0.0001  
min_event_points = int(min_event_duration * sample_rate)

In [ ]:
# Detect indices where current is below threshold
below_threshold = np.where(data < threshold)[0]
if len(below_threshold) == 0:
    print("No events detected with the current threshold.")
    exit()


In [ ]:
# -------------------------------
# 4. Cluster contiguous indices into individual events
# -------------------------------
def cluster_indices(indices, max_gap=5):
    """
    Cluster indices into groups if consecutive indices are separated
    by no more than max_gap samples.
    """
    if len(indices) == 0:
        return []
    
    clusters = []
    current_cluster = [indices[0]]
    
    for i in range(1, len(indices)):
        if indices[i] - indices[i-1] <= max_gap:
            current_cluster.append(indices[i])
        else:
            clusters.append(current_cluster)
            current_cluster = [indices[i]]
    clusters.append(current_cluster)
    return clusters

clusters = cluster_indices(below_threshold)


events = [cluster for cluster in clusters if len(cluster) >= min_event_points]

print(f"Detected {len(events)} events.")

In [ ]:
# ------------------------------- 
# 5. Group contiguous indices into single events
# ------------------------------- 

def group_contiguous(indices, max_gap=1):
    groups = []
    current_group = [indices[0]]
    
    for i in indices[1:]:
        if i - current_group[-1] <= max_gap:
            current_group.append(i)
        else:
            groups.append(current_group)
            current_group = [i]
    groups.append(current_group)
    return groups

clusters = group_contiguous(below_threshold)


event_groups = [cluster for cluster in clusters if len(cluster) >= min_event_points]


In [ ]:
# ---------------------------
# 6. Extract event properties
# ---------------------------
events = []
for group in event_groups:
    start_idx = group[0]
    end_idx = group[-1]
    event_time = time[start_idx:end_idx+1]
    event_data = data[start_idx:end_idx+1]
    
    duration = event_time[-1] - event_time[0]
    amplitude = np.min(event_data)  
    
    events.append({
        "start_time": event_time[0],
        "end_time": event_time[-1],
        "duration": duration,
        "amplitude": amplitude
    })


In [ ]:
# ---------------------------
# 7. Report the detected events
# ---------------------------
print("Detected events:")
for i, event in enumerate(events, start=1):
    print(f"Event {i}: Start = {event['start_time']:.3f} s, "
          f"End = {event['end_time']:.3f} s, Duration = {event['duration']:.3f} s, "
          f"Amplitude = {event['amplitude']:.2f} pA")


In [ ]:
## ---------------------------##
## ONLY VALID FOR SINGLE SWEEP ##
## ---------------------------##

import matplotlib.pyplot as plt
import numpy as np

# Plot the full trace
plt.figure(figsize=(12, 6))
plt.plot(time, signal, color="lightgray", label="Full Trace")

# Use a colormap to assign different colors to each event
colors = plt.cm.viridis(np.linspace(0, 1, len(event_groups)))

# Highlight each event over the full trace
for i, group in enumerate(event_groups):
    plt.plot(time[group], data[group], color=colors[i],
             linewidth=2, label=f"Event {i+1}")

plt.xlabel("Time (s)")
plt.ylabel("Current (pA)")
plt.title("Ionic Current Trace with Highlighted Events")
#plt.legend()
plt.show()


In [ ]:
import pandas as pd

# Create a list to store event information as dictionaries
event_list = []
for i, group in enumerate(event_groups, start=1):
    # Get the indices for this event
    extra_points = 200
    start_idx = group[0] - extra_points
    end_idx = group[-1] + extra_points
    
    # Extract the time and current for this event and convert to lists
    event_time = time[start_idx:end_idx+1].tolist()
    event_current = data[start_idx:end_idx+1].tolist()
    
    ## little extra points
    extra_point = 10
    start_idx = group[0] - extra_point
    end_idx = group[-1] + extra_point
    
    # Extract the time and current for this event and convert to lists
    event_time_l = time[start_idx:end_idx+1].tolist()
    event_current_l = data[start_idx:end_idx+1].tolist()


    # Calculate duration and amplitude (minimum current)
    duration = event_time_l[-1] - event_time_l[0]
    amplitude = min(event_current_l)
    
    # Append the event details to the list
    event_list.append({
        "event_index": i,
        "start_time": event_time[0],
        "end_time": event_time[-1],
        "duration": duration,
        "amplitude": amplitude,
        "time_series": event_time,
        "current_series": event_current,
        "exact_time_series": event_time_l,
        "exact_current_series": event_current_l
    })

# Convert the list of dictionaries into a pandas DataFrame
df_events = pd.DataFrame(event_list)

# (Optional) Save the DataFrame to a CSV file
#df_events.to_csv("detected_events.csv", index=False)

# Display the DataFrame
df_events

In [ ]:
import pandas as pd
df_events = pd.read_csv("detected_events.csv")


In [ ]:
df_events

In [ ]:
import matplotlib.pyplot as plt
import ast

def plot_event(event_idx, events_df):
    """
    Plot an individual event from events_df using its time_series and current_series,
    and mark the event's start and end times with vertical lines.
    
    Parameters:
      event_idx (int): The event index as stored in the "event_index" column.
      events_df (pd.DataFrame): DataFrame containing event data with columns:
                                'event_index', 'start_time', 'end_time',
                                'time_series', and 'current_series'.
    """
    # Retrieve the event row from events_df
    event_row = events_df[events_df["event_index"] == event_idx]
    if event_row.empty:
        print(f"Event index {event_idx} not found in events_df.")
        return
    
    event_row = event_row.iloc[0]
    time_series = event_row["time_series"]        
    current_series = event_row["current_series"]     
    start_time = event_row["start_time"]
    end_time = event_row["end_time"]

    # Convert string representations to lists if needed
    if isinstance(time_series, str):
        time_series = ast.literal_eval(time_series)
    if isinstance(current_series, str):
        current_series = ast.literal_eval(current_series)
    
    # Plot the event's current vs. time
    plt.figure(figsize=(10, 5))
    plt.plot(time_series, current_series, label=f"Event {event_idx}", color="blue")
    
    # Mark the event boundaries with vertical lines:
    #plt.axvline(x=start_time, color="red", linestyle="--", linewidth=2, label="Start Time")
    #plt.axvline(x=end_time, color="green", linestyle="-.", linewidth=2, label="End Time")
    
    plt.xlabel("Time (s)")
    plt.ylabel("Current (A)")
    plt.title(f"Event {event_idx} (Time {start_time} to {end_time})")
    plt.legend()
    plt.show()

# Example usage:
# plot_event(event_idx=150, events_df=df_events)




In [ ]:
plot_event(event_idx=150, events_df=df_events)

### Features

In [ ]:
import numpy as np
import pandas as pd

# Let's assume `event_list` already exists and has the keys:
# "event_index", "start_time", "end_time", "duration", "amplitude",
# "time_series", and "current_series".

for event in event_list:
    # Convert the time and current lists to numpy arrays for easier computation.
    event_time = np.array(event["time_series"])
    event_current = np.array(event["current_series"])
    
    # Compute additional features if data exists.
    if len(event_current) > 0:
        baseline = event_current[0]
        mean_current = np.mean(event_current)
        std_current = np.std(event_current)
        area = np.trapz(event_current, event_time)
        peak_current = np.max(event_current)
    else:
        baseline = np.nan
        mean_current = np.nan
        std_current = np.nan
        area = np.nan
        peak_current = np.nan
    
    # Compute the gradient (slope) only if there are at least two points.
    if len(event_current) > 1:
        slopes = np.gradient(event_current, event_time)
        max_slope = np.max(slopes)
        min_slope = np.min(slopes)
    else:
        max_slope = np.nan
        min_slope = np.nan
    
    n_points = len(event_current)
    
    # Save the features into the event dictionary.
    event["baseline"] = baseline
    event["mean_current"] = mean_current
    event["std_current"] = std_current
    event["area"] = area
    event["peak_current"] = peak_current
    event["max_slope"] = max_slope
    event["min_slope"] = min_slope
    event["n_points"] = n_points

# Convert the updated list of dictionaries into a pandas DataFrame.
df_events = pd.DataFrame(event_list)

In [ ]:
df_events

In [ ]:
import plotly.express as px

# Create an interactive histogram of the 'area' feature
fig = px.histogram(df_events, x="rms_noise", nbins=20,
                   title="Histogram of Event Area",
                   labels={"area": "Area (A.U.)"})
fig.show()
